In [36]:
# import all the modules

import os
import csv
import requests
from zipfile import ZipFile
from io import BytesIO, TextIOWrapper
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt


In [63]:
# grab zip file MHSA dataset
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist()

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        accidents = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')


accidents.head()

C:\Users\Jessie\AppData\Local\Temp\ipykernel_46096\2608785646.py:12: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')


,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,DAYS_RESTRICT,DAYS_LOST,TRANS_TERM,RETURN_TO_WORK_DT,IMMED_NOTIFY_CD,IMMED_NOTIFY,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND
0,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",MPH,220132310045,3,"STRIP, QUARY, OPEN PIT",08/12/2013,...,2.0,0,N,08/19/2013,?,NO VALUE FOUND,NaN,Employee says he slipped off back of fuel truc...,NaN,M
1,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",NaN,220241210019,30,MILL OPERATION/PREPARATION PLANT,04/22/2024,...,0.0,0,N,04/22/2024,?,NO VALUE FOUND,NaN,While blowing lime dust from the lime cooler a...,NaN,M
2,100009,M00024,Legacy Vulcan Corp (Form:Vulcan Materials Co),L16168,"Vulcan Construction Materials, LLC",NaN,220053250019,3,"STRIP, QUARY, OPEN PIT",11/13/2005,...,13.0,0,N,12/01/2005,?,NO VALUE FOUND,NaN,2 employees were attempting to load troughing ...,320053360009,M
3,100011,M11763,Imerys S A,L17074,Imerys Pigments LLC,NaN,220062000052,30,MILL OPERATION/PREPARATION PLANT,07/14/2006,...,0.0,0,N,07/15/2006,?,NO VALUE FOUND,NaN,Employee was pushing against pry bar. Bar slip...,NaN,M
4,100011,M11763,Imerys S A,0090005,Imerys Carbonates LLC,NaN,220090120006,30,MILL OPERATION/PREPARATION PLANT,01/08/2009,...,0.0,0,N,01/09/2009,?,NO VALUE FOUND,NaN,Employee tripped on c-channel at #6 silo. Fel...,NaN,M


## Geospatial 
##### Get address -> make them to lat and lon

In [64]:
# grab zip file for addresses
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/AddressofRecord.zip')

# extract zipfile
with ZipFile(BytesIO(response.content)) as zf:
    # get our text file
    filename = zf.namelist()

    # open text file
    with zf.open(filename[0]) as file:
        # make it a dataframe
        address = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')


address.head()

,MINE_ID,MINE_NAME,CONTACT_TITLE,NEAREST_TOWN,BUSINESS_NAME,STREET,PO_BOX,CITY,STATE_ABBR,FIPS_STATE_CD,STATE,ZIP_CD,COUNTRY,PROVINCE,POSTAL_CD,MINE_TYPE_CD,MINE_STATUS,MINE_STATUS_DT,PRIMARY_SIC_CD,COAL_METAL_IND
0,100003,O'Neal Quarry & Mill,Operations Manager,Calera,"Lhoist North America of Alabama, LLC",2885 Hwy 31 South,NaN,Calera,AL,1.0,Alabama,35040.0,USA,NaN,NaN,Surface,Active,01/22/1979,Lime,M
1,100004,Brierfield Quarry,Quarry Manager,Brierfield,"Lhoist North America of Alabama, LLC",14786 Montevallo Road,NaN,Brierfield,AL,1.0,Alabama,35035.0,USA,NaN,NaN,Surface,Active,03/04/2003,"Crushed, Broken Limestone NEC",M
2,100005,Birmingham Plant,NaN,NaN,Allied Lime Company,NaN,36130,BIRMINGHAM,AL,1.0,Alabama,35236.0,USA,NaN,NaN,Surface,Abandoned,08/15/1989,"Crushed, Broken Limestone NEC",M
3,100006,Auburn Quarry,Safety Representative,Auburn,"Martin Marietta Materials, Inc.",245 Rockwood Rd,NaN,Tyrone,GA,13.0,Georgia,30290.0,USA,NaN,NaN,Surface,Active,09/24/1976,"Crushed, Broken Limestone NEC",M
4,100008,Landmark Plant,Operations Manager,Alabaster,Cheney Lime & Cement Company,1152 Old Hwy 31,NaN,Alabaster,AL,1.0,Alabama,35007.0,USA,NaN,NaN,Surface,Active,11/14/1975,Lime,M


In [74]:
!pip install pgeocode

fix this later

In [ ]:
# from street address -> get lon and lat
import pgeocode

nomi = pgeocode.Nominatim('us')

# zip

# for each location
for index, row in address.iterrows():
    
    # get lat and lon from 
    getLoc = nomi.query_postal_code(str([row['ZIP_CD']]))

    location = {
        'lat' : getLoc['latitude'],
        'lon' : getLoc['longitude']

    }


location.head()

AttributeError: 'dict' object has no attribute 'head'